<h1> Age of Stellar Clusters Project </h1>
<p1> This is the Jupyer Notebook write-up for your semester-long project.  </p1>

<h3 align='center'>Here are the Tables of clusters for your reference.</h3>
<p1 align='center'>Note: Compare the converted RA and DEC values here to your own to verify everything</p1>

In [70]:
from __future__ import division
import hide_toggle
import pandas as pd
import cluster as cl
import numpy as np
from IPython.display import HTML
from IPython.display import display
from IPython.display import display_pretty
from ipywidgets import interact, interactive, fixed, interact_manual
from matplotlib.collections import LineCollection
from matplotlib import colors as mcolors
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib

display(cl.open_cluster_out)

display(cl.globular_cluster_out)

hide_toggle.hide_toggle()


,Name,RA (J2000),DEC (J2000),RA (DEG),DEC (DEG),Distance (pc)
0,NGC 3293,10 35 49,-58 13 48,158.954,-58.23,2750
1,M 11,18 51 05,-06 16 12,282.771,-6.27,1900
2,M 26,18 45 18,-09 23 00,281.325,-9.38333,1533
3,M 35,06 08 54,+24 20 00,92.225,24.3333,858
4,M 44,08 40 24,+19 40 00,130.1,19.6667,177
5,M 45,03 47 00,+24 07 00,56.75,24.1167,135
6,M 46,07 41 46,-14 48 36,115.442,-14.81,1656


,Name,RA (J2000),DEC (J2000),RA (DEG),DEC (DEG),Distance (pc)
0,47 Tuc,00 24 05.4,-72 04 53.2,6.0225,-72.0814,4001
1,M 2,21 33 27.0,-00 49 23.7,323.363,-0.82325,16863
2,M 3,13 42 11.6,+28 22 38.2,205.548,28.3773,10400
3,M 5,15 18 33.2,+02 04 51.7,229.638,2.08103,7499
4,M 13,16 41 41.6,+36 27 40.8,250.423,36.4613,6800
5,M 14,17 37 36.2,-03 14 45.3,264.401,-3.24592,9290
6,M 30,21 40 22.1,-23 10 47.5,325.092,-23.1799,8300


<h2> Cluster Plotting </h2>
<p1>Your first task is to develop plots for the clusters of your choice. Remember, you need to choose <b>(1) Open Cluster</b> and also <b>(1) Globular Cluster</b>! Enter the names of your clusters of choice below: (<i><u>Remember to re-run the code block below the pull-down selection boxes if you change your selection</u></i>)

In [3]:
def open_clus(Open_Cluster):
    return Open_Cluster
def glob_clus(Globular_Cluster):
    return Globular_Cluster
pl_open = interactive(open_clus, Open_Cluster = {cl.open_clusters[i].name:cl.open_clusters[i] for i in range(len(cl.open_clusters))})
pl_glob = interactive(glob_clus, Globular_Cluster = {cl.globular_clusters[i].name:cl.globular_clusters[i] for i in range(len(cl.globular_clusters))})
display(pl_open)
display(pl_glob)
hide_toggle.hide_toggle()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUnT3Blbl9DbHVzdGVyJywgb3B0aW9ucz17J00gMzUnOiA8Y2x1c3Rlci5jbHVzdGVyIGluc3RhbmNlIGF0IDB4MTHigKY=


aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUnR2xvYnVsYXJfQ2x1c3RlcicsIG9wdGlvbnM9eydNIDUnOiA8Y2x1c3Rlci5jbHVzdGVyIGluc3RhbmNlIGF0IDDigKY=


<h3>Matching</h3>
<p1>Now that you have selected the clusters you are going to use for your project, you need to find all of the stars that are actually in the cluster. To do this, you will first define a distance threshold from the cluster center, in which all stars are considered to be a part of the cluster. Dr. Kirkpatrick recommends that you start with 15' and adjust from there.</p1>

In [23]:
def dist_thresh(Dist_Threshold_Open=15.0, Dist_Threshold_Globular=15.0):
    return Dist_Threshold_Open, Dist_Threshold_Globular
pl_dist_thresh = interactive(dist_thresh, Dist_Threshold_Open = (0.0, 15.0), Dist_Threshold_Globular = (0.0,15.0))
display(pl_dist_thresh)
hide_toggle.hide_toggle()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPTE1LjAsIGRlc2NyaXB0aW9uPXUnRGlzdF9UaHJlc2hvbGRfT3BlbicsIG1heD0xNS4wKSwgRmxvYXRTbGlkZXIodmFsdWXigKY=


<p1>Now we will load in the .csv file with all of the stars, and will print out a few lines from catalog so you can get a picture of how the data is formatted.</p1>

In [146]:
#This table contains all of the star data
all_data = pd.read_csv('cluster_cat.csv',delimiter=',')
display(all_data[0:5]) #Display the first 5 stars
hide_toggle.hide_toggle()

,source_id,ra,dec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag
0,4252502065373003776,282.745453,-6.326164,17.559374,17.854786,16.482748
1,4252499552740270336,282.795034,-6.320462,18.747301,18.879366,17.172401
2,4252499076075131264,282.792084,-6.349008,20.261185,NaN,NaN
3,4252502168452220416,282.756594,-6.316115,19.334372,18.537830,18.081583
4,4252502060994791808,282.739702,-6.321722,14.084452,14.496042,13.489761


<h3>Determining Distance from Cluster</h3>
<p1>Your next task is to write a function to calculate the distance from a star to a cluster. The Function below has already been laid out for you. You should need the star's RA and DEC values (star_RA and star_DEC, respectively) along with the cluster RA and DEC values (clus_RA and clus_DEC, respectively) to write this function. The function should <u> return the distance to the cluster</u>. Use what you learned in class about coordinate systems to write this out, and remember to <b>keep track of your units! </b>

In [159]:
def distance_from_cluster(star_RA, star_DEC, clus_RA, clus_DEC):
    #avg_DEC = (clus_DEC + star_DEC)/2
    #delta_RA = (clus_RA - star_RA)*np.cos(avg_DEC*np.pi/180)
    delta_RA = (clus_RA - star_RA)*np.cos(np.deg2rad(star_DEC))
    delta_DEC = clus_DEC-star_DEC
    return np.sqrt( delta_RA**2 + delta_DEC**2)

In [160]:
###
## These lines read in your entries into the cluster names and distance threshold boxes
###
Open_cluster = pl_open.result
Globular_cluster = pl_glob.result
distance_threshold_open = pl_dist_thresh.result[0]
distance_threshold_glob = pl_dist_thresh.result[1]

dist_thresh_open_deg = float(distance_threshold_open)/60
dist_thresh_glob_deg = float(distance_threshold_open)/60
###

###
## Let us define the cluster locations to easy to remember variables. Note that these are in degrees
###
open_cluster_RA , open_cluster_DEC = [float(val) for val in Open_cluster.get_position_degrees()]

globular_cluster_RA, globular_cluster_DEC = [float(val) for val in Globular_cluster.get_position_degrees()]
###


stars_inOpen = []
stars_inGlobular = []

N_stars = len(all_data)
for star_index in range(N_stars):
    
    #Let us save this star's position to some variables.
    star_RA = all_data.at[star_index, 'ra']
    star_DEC = all_data.at[star_index, 'dec']
    
    dist_from_open_cluster = distance_from_cluster(star_RA, star_DEC, open_cluster_RA, open_cluster_DEC)
    
    dist_from_globular_cluster = distance_from_cluster(star_RA, star_DEC, globular_cluster_RA, globular_cluster_DEC)
    
    ###
    
    #Now we check to see if this star is in either cluster
    if dist_from_open_cluster <= dist_thresh_open_deg:
        stars_inOpen.append(star_index)
    if dist_from_globular_cluster <= dist_thresh_glob_deg:
        stars_inGlobular.append(star_index)
    
hide_toggle.hide_toggle()

<h2>Cluster Plots</h2>
<p1>Now we know whether each star is in either cluster, so we can go ahead and make plots of the stars in each cluster within the threshold distance you set.</p1>


 <h3>Open Cluster Plot</h3>

In [161]:
%config InlineBackend.figure_format = 'svg'
params = {
    'text.latex.preamble': ['\\usepackage{gensymb}'],
    'image.origin': 'lower',
    'image.interpolation': 'nearest',
    'image.cmap': 'gray', #gray
    'axes.grid': False,
    'savefig.dpi': 300,  # to adjust notebook inline plot size
    'axes.labelsize': 16, # fontsize for x and y labels (was 10); 40
    'axes.titlesize': 20, #40
    'font.size': 28, # was 10 #28
    'legend.fontsize': 30, # was 10; 28
    'xtick.labelsize': 14, #28
    'ytick.labelsize': 14, #28
    'text.usetex': True,
    'figure.figsize': [6, 6], #4.49, 3.37
    'figure.autolayout': True,
    'font.family': 'monospace',
}
matplotlib.rcParams.update(params)
Open_cluster = pl_open.result

y_vals_open = all_data.iloc[stars_inOpen,:]['phot_rp_mean_mag']
x_vals_open = all_data.iloc[stars_inOpen,:]['phot_bp_mean_mag'] - all_data.iloc[stars_inOpen,:]['phot_rp_mean_mag']


def func_open(x_min, x_max, y_min, y_max, show_title):
    fig, ax = plt.subplots()
    ax.set_xlabel(r'$G_{BP}-G_{RP}$')
    ax.set_ylabel(r'$G_{RP}$')
    ax.grid(which= 'major', linestyle='-', linewidth='0.5', color = 'black', alpha = 0.3)
    ax.grid(which='minor', linestyle=':', linewidth='0.3', color = 'black', alpha = 0.2)
    if show_title:
        ax.set_title(r'$'+Open_cluster.name+'$')
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([y_max, y_min])
    
    ax.scatter(x_vals_open, y_vals_open, marker = 'x', c='k')
    plt.show()
    
plot_open = interact_manual(func_open, x_max=1.2*np.nanmax(x_vals_open), x_min=(np.nanmin(x_vals_open)-5*abs(np.nanmin(x_vals_open)))/5, y_max = 1.2*np.nanmax(y_vals_open), y_min =(np.nanmin(y_vals_open)-6*abs(np.nanmin(y_vals_open)))/5, show_title= True)
#display(plot_open)

hide_toggle.hide_toggle()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPS0wLjMzMzE4MDAwMDAwMDAwMDE0LCBkZXNjcmlwdGlvbj11J3hfbWluJywgbWF4PTAuMzMzMTgwMDAwMDAwMDAwMTQsIG3igKY=


 <h3>Globular Cluster Plot</h3>

In [162]:
y_vals_glob = all_data.iloc[stars_inGlobular,:]['phot_rp_mean_mag']
x_vals_glob = all_data.iloc[stars_inGlobular,:]['phot_bp_mean_mag'] - all_data.iloc[stars_inGlobular,:]['phot_rp_mean_mag']


def func_glob(x_min, x_max, y_min, y_max, show_title):
    fig, ax = plt.subplots()
    ax.set_xlabel(r'$G_{BP}-G_{RP}$')
    ax.set_ylabel(r'$G_{RP}$')
    ax.grid(which= 'major', linestyle='-', linewidth='0.5', color = 'black', alpha = 0.3)
    ax.grid(which='minor', linestyle=':', linewidth='0.3', color = 'black', alpha = 0.2)
    if show_title:
        ax.set_title(r'$'+Globular_cluster.name+'$')
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([y_max, y_min])
    
    ax.scatter(x_vals_glob, y_vals_glob, marker = 'x', c='k')
    plt.show()
    
plot_glob = interact_manual(func_glob, x_max=1.2*np.max(x_vals_glob), x_min=(np.min(x_vals_glob)-6*abs(np.min(x_vals_glob)))/5, y_max = 1.2*np.max(y_vals_glob), y_min =(np.min(y_vals_glob)-6*abs(np.min(y_vals_glob)))/5, show_title = True)
#display(plot_glob)

hide_toggle.hide_toggle()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPS0wLjA0NTM3Nzk5OTk5OTk5OTQ3NCwgZGVzY3JpcHRpb249dSd4X21pbicsIG1heD0wLjA0NTM3Nzk5OTk5OTk5OTQ3NCzigKY=


Now, with our general plots made, we will begin our work into fitting these plots with isochrones to determine the cluster age. First, we must convert our y-axis to be the absolute magnitude in the G_RP band. Write a function for this below:

In [163]:
#Write a function to return the absolute magnitude of a star knowing its apparent magnitude, m, and its distance, d, in pc
def absolute_M(m, dist):
    M = m - 5*np.log10(dist/10)
    
    return M

<p1>Now, using the function you wrote for the absolute magnitude, we will convert our previous plots to display this, and we will also overlay the isochrones which we will later fit.</p1>

<h3>Open Cluster: Absolute Magnitude</h3>


In [164]:
#Isochrone file:
# col1: logAge [Gyr]
# col2: stellar mass [M star]
# col3: Luminosity
# col4: Temperature
# col5: G
# col6: G_BP
# col7: G_RP
isochrones = np.loadtxt("isochrones.txt")
iso_GRP = isochrones[:,6]
iso_GBP_GRP = isochrones[:,5]-isochrones[:,6]


lines = []
age_list = []
num_iso = len(isochrones)
age_count = isochrones[0][0]
start_index = 0

for i in range(num_iso):
    age_current = isochrones[i][0]
    if age_current != age_count:
        age_count = age_current
        age_list.append(age_current)
        lines.append(zip(iso_GBP_GRP[start_index:(i-1)],iso_GRP[start_index:(i-1)]) )
        start_index = i



y_vals_open_abs = absolute_M(all_data.iloc[stars_inOpen,:]['phot_rp_mean_mag'], Open_cluster.dist)

def func_open_abs(x_min, x_max, y_min, y_max, show_title):
    line_segments = LineCollection(lines,linestyles='solid', color = 'r', lw = (0.5))
    fig, ax = plt.subplots()
    ax.set_xlabel(r'$G_{BP}-G_{RP}$')
    ax.set_ylabel(r'$M_{G_{RP}}$')
    ax.grid(which= 'major', linestyle='-', linewidth='0.5', color = 'black', alpha = 0.3)
    ax.grid(which='minor', linestyle=':', linewidth='0.3', color = 'black', alpha = 0.2)
    if show_title:
        ax.set_title(r'$'+Open_cluster.name+'$')
    ax.set_xlim([float(x_min), float(x_max)])
    ax.set_ylim([float(y_max), float(y_min)])
    
    ax.scatter(x_vals_open, y_vals_open_abs, marker = 'x', c='k')
    
    #ax.plot(lines[-1], c = 'r', lw = 0.5)
    ax.add_collection(line_segments)
    
    plt.show()


plot_open = interact_manual(func_open_abs, x_max=str(1.2*np.nanmax(x_vals_open)), x_min=str(np.min(x_vals_open)), y_max = str(1.2*np.nanmax(y_vals_open_abs)), y_min =str(np.min(y_vals_open_abs)), show_title= True)
#display(plot_open)
hide_toggle.hide_toggle()


aW50ZXJhY3RpdmUoY2hpbGRyZW49KFRleHQodmFsdWU9dScwLjQxNjQ3NTAwMDAwMDAwMDE1JywgZGVzY3JpcHRpb249dSd4X21pbicpLCBUZXh0KHZhbHVlPXUnMi45NDY2NzMyMDAwMDAwMDPigKY=


<h3>Globular Cluster: Absolute Magnitude</h3>

In [165]:
y_vals_glob_abs = absolute_M(all_data.iloc[stars_inGlobular,:]['phot_rp_mean_mag'], Globular_cluster.dist)

def func_glob_abs(x_min, x_max, y_min, y_max, show_title):
    line_segments = LineCollection(lines,linestyles='solid', color = 'r', lw = (0.5))
    fig, ax = plt.subplots()
    ax.set_xlabel(r'$G_{BP}-G_{RP}$')
    ax.set_ylabel(r'$G_{RP}$')
    ax.grid(which= 'major', linestyle='-', linewidth='0.5', color = 'black', alpha = 0.3)
    ax.grid(which='minor', linestyle=':', linewidth='0.3', color = 'black', alpha = 0.2)
    if show_title:
        ax.set_title(r'$'+Globular_cluster.name+'$')
    ax.set_xlim([float(x_min), float(x_max)])
    ax.set_ylim([float(y_max), float(y_min)])
    
    ax.scatter(x_vals_glob, y_vals_glob_abs, marker = 'x', c='k')
    ax.add_collection(line_segments)
    plt.show()
    
plot_glob = interact_manual(func_glob_abs, x_max=str(1.2*np.nanmax(x_vals_glob)), x_min=str(np.min(x_vals_glob)), y_max = str(1.2*np.nanmax(y_vals_glob_abs)), y_min =str(np.min(y_vals_glob_abs)), show_title = True)
#display(plot_glob)

hide_toggle.hide_toggle()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KFRleHQodmFsdWU9dScwLjA0NTM3Nzk5OTk5OTk5OTQ3NCcsIGRlc2NyaXB0aW9uPXUneF9taW4nKSwgVGV4dCh2YWx1ZT11JzIuMjg5ODY5OTk5OTk5OTnigKY=
